In [1]:
from flask import Flask,request, url_for, redirect, render_template, jsonify
from pycaret.regression import *
import pandas as pd
import pickle
import numpy as np

########
import dash 
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

In [2]:
# Loads pre-trained model
loaded_model = pickle.load(open('model.sav', 'rb'))

In [3]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="login"
)

print(mydb)

mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM login")

myresult = mycursor.fetchall()

dec = []
key = 5

import string
 
all_letters= string.ascii_letters

for x in myresult:
    dict2 = {}    
    for i in range(len(all_letters)):
        dict2[all_letters[i]] = all_letters[(i-key)%(len(all_letters))]

    # loop to recover plain text
    decrypt_txt = []

    s = x[1]

    for char in s:
        if char in all_letters:
            temp = dict2[char]
            decrypt_txt.append(temp)
        else:
            temp = char
            decrypt_txt.append(temp)

    decrypt_txt = "".join(decrypt_txt)
    dec.append([x[0],decrypt_txt])
    
dec = dict(dec)
dec

{'1832003': 'adithyar',
 '1832006': 'amrishkn',
 '1832027': 'kaalishwarr',
 '1832032': 'manojp',
 '1832048': 'shreyasd'}

In [4]:
# from dash import Dash, dcc, html
# from dash.dependencies import Input, Output

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.QUARTZ])
app.title = 'Heart Attack Predictor'

import dash_auth
VALID_USERNAME_PASSWORD_PAIRS = dec

auth = dash_auth.BasicAuth(
    app,
    VALID_USERNAME_PASSWORD_PAIRS
)


ALLOWED_TYPES = (
  'Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS','RestingECG','MaxHR', 'ExerciseAngina', 'Oldpeak','ST_Slope'
)


app.layout = html.Div(
    [
        dbc.Row(dbc.Col(html.H3("Heart Attack Predictor"),
                        width={'size': 6, 'offset': 4},
                        ),
                ),
        dbc.Row(
            children = [
            dbc.Col( children = [
        dbc.Label("Age"),
        dbc.Input(placeholder="age of the patient [years]", type="text", id = 'c1'),
       # dbc.FormText("age of the patient [years]"),
        ]), 
            
            dbc.Col( children = [
        dbc.Label("Sex"),
        dbc.Input(placeholder="sex of the patient [M: Male, F: Female]", type="text", id = 'c2'),
        #dbc.FormText("sex of the patient [M: Male, F: Female]"),
        ]),
            ]),
               
        dbc.Row(
            children = [
            dbc.Col( children = [
        dbc.Label("ChestPainType"),
        dbc.Input(placeholder="TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic", type="text", id = 'c3'),
       # dbc.FormText("[TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]"),
       
        ]),
              
        
            dbc.Col( children = [
        dbc.Label("RestingBP"),
        dbc.Input(placeholder="resting blood pressure [mm Hg]", type="text", id = 'c4'),
       # dbc.FormText("resting blood pressure [mm Hg]"),
        ]),
            ]),
        
        dbc.Row(
            children = [
            dbc.Col( children = [
        dbc.Label("Cholesterol"),
        dbc.Input(placeholder="serum cholesterol [mm/dl]", type="text", id = 'c5'),
        #dbc.FormText("serum cholesterol [mm/dl]"),
        ]),
    
            dbc.Col( children = [
        dbc.Label("Fasting Blood Sugar"),
        dbc.Input(placeholder="fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]", type="text", id = 'c6'),
      #  dbc.FormText("fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]"),
        ]),              
            ]),
        
        dbc.Row(
            children = [
            dbc.Col( children = [
        dbc.Label("RestingECG"),
        dbc.Input(placeholder="Normal/ST/LVH", type="text", id = 'c7'),
        #dbc.FormText("M/F"),
        ]),
           
            dbc.Col( children = [
        dbc.Label("MaxHR"),
        dbc.Input(placeholder="maximum heart rate achieved [Numeric value between 60 and 202]", type="text", id = 'c8'),
        #dbc.FormText("M/F"),
        ]),
            ]),
        
        dbc.Row(children = [
            dbc.Col( children = [
        dbc.Label("ExerciseAngina"),
        dbc.Input(placeholder="exercise-induced angina [Y: Yes, N: No]", type="text", id = 'c9'),
        #dbc.FormText("Types : ASY, ATA, NAP, TY"),
        ]),
            
            dbc.Col( children = [
        dbc.Label("Oldpeak"),
        dbc.Input(placeholder="oldpeak = ST [Numeric value measured in depression]", type="text", id = 'c10'),
        ]),
            ]),
        
        html.Hr(),
        
        dbc.Row(
            dbc.Button("PREDICT", id = 'btn-submit'),
           # width={'size': 6, "offset": 3}   
        ),
       html.Hr(),
            
        dbc.Row(dbc.Col( 
            dbc.Label("Heart Attack Possibility :"),
        ),
               ),    
        
        html.Div(id='output-submit'),
             
    ])

@app.callback(Output('output-submit', 'children'),
                [Input('btn-submit', 'n_clicks')],
                [State('c1', 'value'),State('c2', 'value'),State('c3', 'value'),State('c4', 'value'),State('c5', 'value'),State('c6', 'value'),State('c7', 'value'),State('c8', 'value'),State('c9', 'value'),State('c10', 'value')])


def update_output(clicked, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10):
    if clicked:
        x = [0 for i in range(15)]
        x[0] = c1
        x[1] = 1 if c2 == 'M' else 0
        
        if c3 == 'TA':
            x[11] = 1
        elif c3 == 'ATA':
            x[9] = 1
        elif c3 == 'NAP':
            x[10] = 1
        elif c3 == 'ASY':
            x[8] = 1
        
        x[2] = c4
        x[3] = c5
        x[4] = c6
        
        if c7 == 'Normal':
            x[13] = 1
        elif c7 == 'ST':
            x[14] = 1
        elif c7 == 'LVH':
            x[12] = 1
            
        x[5] = c8
        x[6] = 1 if c9 == 'Y' else 0
        x[7] = c10
        
        iv = pd.DataFrame(x).to_numpy().reshape(1,-1)
        
        pred = loaded_model.predict(iv)
        
        res = "Possibility of Heart Attack is there! Be Careful." if pred == 1 else "Hooray! No possibilty of Heart Attack"
        return res

if __name__ == "__main__":
    app.run_server()
    


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Aug/2022 14:36:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2022 14:36:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2022 14:36:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2022 14:36:41] "GET /_favicon.ico?v=2.3.1 HTTP/1.1" 200 -
127.0.0.1 - - [01/Aug/2022 14:36:41] "POST /_dash-update-component HTTP/1.1" 200 -
